# Benchmarking: gt4py

Timing a gt4py-powered implementation of some basic mathematical operations.

## User input

In [105]:
import numpy as np

# gt4py
backend = "gtcuda"
dtype = np.float64
storage_shape = (321, 321, 120)
default_origin = (3, 3, 0)

# serialization
serialize = True
filename = "timings_daint.xlsx"
sheetname = "{} x {} x {}".format(*storage_shape)

## Data initialization

In [3]:
# install tasmania
!pip install -e ../

# install gt4py
!pip install -e ../docker/external/gt4py[cuda101]

Obtaining file:///users/subbiali/jupyterhub/tasmania-prognostic-saturation
  Using cached https://files.pythonhosted.org/packages/a1/4e/c42167ba5c3192bed633726d39d7896cc55d4fa3ec4a1fb60cd3a53fc4c7/decorator-4.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c2/46/fab473f54ffbb932d191ca96a71a5d4f900b480526466e96e1c0069e7dac/graphviz-0.5.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/44/28/86488fe20a146241ece4570af94b49689ab80b66a1609256a8469a0bd1b0/matplotlib-2.0.0-1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/39/ea/280d6c0d92f8e3ca15fd798bbcc2ea141489f9539de7133d8fe10ea4b049/nbconvert-5.3.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/74/e50234bc82c553fecdbd566d8650801e3fe2d6d8c8d940638e3d8a7c5522/pandas-0.24.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/40/f8/926538c3f358a52ea5fd61bfe78f9c208bfd5ebc152d1

In [106]:
from copy import deepcopy
from gt4py import gtscript, storage as gt_storage
from tasmania.python.utils.gtscript_utils import (
    set_annotations,
    stencil_abs_defs,
    stencil_iabs_defs,
    stencil_copy_defs,
    stencil_copychange_defs,
    stencil_add_defs,
    stencil_iadd_defs,
    stencil_sub_defs,
    stencil_isub_defs,
    stencil_mul_defs,
    stencil_imul_defs,
    stencil_scale_defs,
    stencil_iscale_defs,
    stencil_addsub_defs,
    stencil_iaddsub_defs,
    stencil_fma_defs,
    stencil_sts_rk2_0_defs,
    stencil_sts_rk3ws_0_defs,
)
from tasmania.python.utils.storage_utils import zeros

a = zeros(storage_shape, backend, dtype, default_origin)
a[...] = np.random.rand(*storage_shape)
a_dc = deepcopy(a)
b = zeros(storage_shape, backend, dtype, default_origin)
b[...] = np.random.rand(*storage_shape)
c = zeros(storage_shape, backend, dtype, default_origin)
c[...] = np.random.rand(*storage_shape)
d = zeros(storage_shape, backend, dtype, default_origin)

f = np.random.rand(1).item()

## Serialization setup

In [107]:
# install and import openpyxl
is_installed = !pip list 2> /dev/null | grep 'openpyxl'
if not is_installed:
    !pip install openpyxl==2.6
import openpyxl as xl

# install and import pandas
is_installed = !pip list 2> /dev/null | grep 'pandas'
if not is_installed:
    !pip install pandas
import pandas as pd

# install xlrd
is_installed = !pip list 2> /dev/null | grep 'xlrd'
if not is_installed:
    !pip install xlrd

In [108]:
# dataframe's indices and column
index = [
    "copy", 
    "copychange", 
    "abs", 
    "iabs", 
    "add", 
    "iadd", 
    "sub", 
    "isub", 
    "mul", 
    "imul",
    "addsub",
    "iaddsub",
    "fma",
    "sts_rk2_0",
    "sts_rk3ws_0"
]
if backend == "debug":
    column = "gtdebug"
elif backend == "numpy":
    column = "gtnumpy"
else:
    column = backend
    
if serialize:
    # create the spreadsheet
    import os
    if not os.path.exists(filename):
        wb = xl.Workbook()
        wb.save(filename=filename)
    else:
        wb = xl.load_workbook(filename)
        
    # create an ExcelWriter object
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        writer.book = wb
        # writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)
    
    if sheetname not in wb.sheetnames:
        # create an empty dataframe
        df = pd.DataFrame(data=[None,]*len(index), index=index, columns=[column,])
        df.to_excel(writer, sheet_name=sheetname)
    else:
        # load the dataframe
        df = pd.read_excel(writer, sheet_name=sheetname, index_col=0)
        
        # remove the sheet
        sheetid = wb.sheetnames.index(sheetname)
        wb.remove(wb.worksheets[sheetid])
else:
    # create the dataframe
    df = pd.DataFrame(data=[None,]*len(index), index=index, columns=[column,])

## Timing

In [109]:
# copy
set_annotations(stencil_copy_defs, dtype)
stencil_copy = gtscript.stencil(backend=backend, definition=stencil_copy_defs, rebuild=False)
out = %timeit -o stencil_copy(src=b, dst=a, origin=(0, 0, 0), domain=storage_shape)
df.at["copy", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_copy_defs__gtcuda_c1058d5bcb_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copy_defs__gtcuda_c1058d5bcb_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copy_defs__gtcuda_c1058d5bcb_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copy_defs__gtcuda_c1058d5bcb_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copy_defs__gtcuda_c1058d5bcb_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copy_defs__gtcuda_c

In [110]:
# copychange
set_annotations(stencil_copychange_defs, dtype)
stencil_copychange = gtscript.stencil(backend=backend, definition=stencil_copychange_defs, rebuild=False)
out = %timeit -o stencil_copychange(src=b, dst=a, origin=(0, 0, 0), domain=storage_shape)
df.at["copychange", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_copychange_defs__gtcuda_bdf8e4770e_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copychange_defs__gtcuda_bdf8e4770e_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copychange_defs__gtcuda_bdf8e4770e_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copychange_defs__gtcuda_bdf8e4770e_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_copychange_defs__gtcuda_bdf8e4770e_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__

In [111]:
# abs
set_annotations(stencil_abs_defs, dtype)
stencil_abs = gtscript.stencil(backend=backend, definition=stencil_abs_defs, rebuild=False)
out = %timeit -o stencil_abs(a, b, origin=(0, 0, 0), domain=storage_shape)
df.at["abs", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_abs_defs__gtcuda_67b485c90e_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_abs_defs__gtcuda_67b485c90e_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_abs_defs__gtcuda_67b485c90e_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_abs_defs__gtcuda_67b485c90e_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_abs_defs__gtcuda_67b485c90e_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_abs_defs__gtcuda_67b485c

In [112]:
# abs: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_abs(a, b, origin=default_origin, domain=domain)

660 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [113]:
# iabs
a[...] = a_dc[...]
set_annotations(stencil_iabs_defs, dtype)
stencil_iabs = gtscript.stencil(backend=backend, definition=stencil_iabs_defs, rebuild=False)
out = %timeit -o stencil_iabs(a, origin=(0, 0, 0), domain=storage_shape)
df.at["iabs", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_iabs_defs__gtcuda_608b64cd31_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iabs_defs__gtcuda_608b64cd31_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iabs_defs__gtcuda_608b64cd31_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iabs_defs__gtcuda_608b64cd31_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iabs_defs__gtcuda_608b64cd31_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iabs_defs__gtcuda_6

In [114]:
# iabs: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_iabs(a, origin=default_origin, domain=domain)

557 µs ± 984 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [115]:
# add
set_annotations(stencil_add_defs, dtype)
stencil_add = gtscript.stencil(backend=backend, definition=stencil_add_defs, rebuild=False)
out = %timeit -o stencil_add(in_a=a, in_b=b, out_c=c, origin=(0, 0, 0), domain=storage_shape)
df.at["add", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_add_defs__gtcuda_14fbd2645f_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_add_defs__gtcuda_14fbd2645f_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_add_defs__gtcuda_14fbd2645f_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_add_defs__gtcuda_14fbd2645f_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_add_defs__gtcuda_14fbd2645f_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_add_defs__gtcuda_14fbd26

In [116]:
# add: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_add(in_a=a, in_b=b, out_c=c, origin=default_origin, domain=domain)

908 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [117]:
# iadd
a[...] = a_dc[...]
set_annotations(stencil_iadd_defs, dtype)
stencil_iadd = gtscript.stencil(backend=backend, definition=stencil_iadd_defs, rebuild=False)
out = %timeit -o stencil_iadd(inout_a=a, in_b=b, origin=(0, 0, 0), domain=storage_shape)
df.at["iadd", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_iadd_defs__gtcuda_e9d894558c_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iadd_defs__gtcuda_e9d894558c_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iadd_defs__gtcuda_e9d894558c_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iadd_defs__gtcuda_e9d894558c_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iadd_defs__gtcuda_e9d894558c_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iadd_defs__gtcuda_e

In [118]:
# iadd: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_iadd(inout_a=a, in_b=b, origin=default_origin, domain=domain)

810 µs ± 322 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [119]:
# sub
set_annotations(stencil_sub_defs, dtype)
stencil_sub = gtscript.stencil(backend=backend, definition=stencil_sub_defs, rebuild=False)
out = %timeit -o stencil_sub(in_a=a, in_b=b, out_c=c, origin=(0, 0, 0), domain=storage_shape)
df.at["sub", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_sub_defs__gtcuda_2f238c1a6a_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sub_defs__gtcuda_2f238c1a6a_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sub_defs__gtcuda_2f238c1a6a_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sub_defs__gtcuda_2f238c1a6a_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sub_defs__gtcuda_2f238c1a6a_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sub_defs__gtcuda_2f238c1

In [120]:
# sub: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_sub(in_a=a, in_b=b, out_c=c, origin=default_origin, domain=domain)

909 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [121]:
# isub
a[...] = a_dc[...]
set_annotations(stencil_isub_defs, dtype)
stencil_isub = gtscript.stencil(backend=backend, definition=stencil_isub_defs, rebuild=False)
out = %timeit -o stencil_isub(inout_a=a, in_b=b, origin=(0, 0, 0), domain=storage_shape)
df.at["isub", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_isub_defs__gtcuda_8873429266_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_isub_defs__gtcuda_8873429266_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_isub_defs__gtcuda_8873429266_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_isub_defs__gtcuda_8873429266_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_isub_defs__gtcuda_8873429266_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_isub_defs__gtcuda_8

In [122]:
# isub: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_isub(inout_a=a, in_b=b, origin=default_origin, domain=domain)

813 µs ± 578 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [123]:
# mul
set_annotations(stencil_mul_defs, dtype)
stencil_mul = gtscript.stencil(backend=backend, definition=stencil_mul_defs, rebuild=False)
out = %timeit -o stencil_mul(in_a=a, in_b=b, out_c=c, origin=(0, 0, 0), domain=storage_shape)
df.at["mul", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_mul_defs__gtcuda_243139dfd7_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_mul_defs__gtcuda_243139dfd7_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_mul_defs__gtcuda_243139dfd7_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_mul_defs__gtcuda_243139dfd7_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_mul_defs__gtcuda_243139dfd7_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_mul_defs__gtcuda_243139d

In [124]:
# mul: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_mul(in_a=a, in_b=b, out_c=c, origin=default_origin, domain=domain)

906 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [125]:
# imul
a[...] = a_dc[...]
set_annotations(stencil_imul_defs, dtype)
stencil_imul = gtscript.stencil(backend=backend, definition=stencil_imul_defs, rebuild=False)
out = %timeit -o stencil_imul(inout_a=a, in_b=b, origin=(0, 0, 0), domain=storage_shape)
df.at["imul", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_imul_defs__gtcuda_bcb9ed5103_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_imul_defs__gtcuda_bcb9ed5103_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_imul_defs__gtcuda_bcb9ed5103_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_imul_defs__gtcuda_bcb9ed5103_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_imul_defs__gtcuda_bcb9ed5103_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_imul_defs__gtcuda_b

In [126]:
# imul: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_imul(inout_a=a, in_b=b, origin=default_origin, domain=domain)

810 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [127]:
# addsub
set_annotations(stencil_addsub_defs, dtype)
stencil_addsub = gtscript.stencil(backend=backend, definition=stencil_addsub_defs, rebuild=False)
out = %timeit -o stencil_addsub(in_a=a, in_b=b, in_c=c, out_d=d, origin=(0, 0, 0), domain=storage_shape)
df.at["addsub", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_addsub_defs__gtcuda_e36e6a2b52_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_addsub_defs__gtcuda_e36e6a2b52_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_addsub_defs__gtcuda_e36e6a2b52_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_addsub_defs__gtcuda_e36e6a2b52_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_addsub_defs__gtcuda_e36e6a2b52_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_addsub_de

In [128]:
# addsub: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_addsub(in_a=a, in_b=b, in_c=c, out_d=d, origin=default_origin, domain=domain)

1.17 ms ± 705 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [129]:
# iaddsub
a[...] = a_dc[...]
stencil_iaddsub = gtscript.stencil(backend=backend, definition=stencil_iaddsub_defs, rebuild=False)
out = %timeit -o stencil_iaddsub(inout_a=a, in_b=b, in_c=c, origin=(0, 0, 0), domain=storage_shape)
df.at["iaddsub", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_iaddsub_defs__gtcuda_5fd43daecc_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iaddsub_defs__gtcuda_5fd43daecc_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iaddsub_defs__gtcuda_5fd43daecc_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iaddsub_defs__gtcuda_5fd43daecc_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iaddsub_defs__gtcuda_5fd43daecc_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_iadd

In [130]:
# iaddsub: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_iaddsub(inout_a=a, in_b=b, in_c=c, origin=default_origin, domain=domain)

1.08 ms ± 480 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [131]:
# fma
set_annotations(stencil_fma_defs, dtype)
stencil_fma = gtscript.stencil(backend=backend, definition=stencil_fma_defs, rebuild=False)
out = %timeit -o stencil_fma(in_a=a, in_b=b, out_c=c, f=f, origin=(0, 0, 0), domain=storage_shape)
df.at["fma", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_fma_defs__gtcuda_c205107450_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_fma_defs__gtcuda_c205107450_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_fma_defs__gtcuda_c205107450_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_fma_defs__gtcuda_c205107450_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_fma_defs__gtcuda_c205107450_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_fma_defs__gtcuda_c205107

In [132]:
# fma: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_fma(in_a=a, in_b=b, out_c=c, f=f, origin=default_origin, domain=domain)

914 µs ± 708 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [133]:
# sts_rk2_0
set_annotations(stencil_sts_rk2_0_defs, dtype)
stencil_sts_rk2_0 = gtscript.stencil(backend=backend, definition=stencil_sts_rk2_0_defs, rebuild=False)
out = %timeit -o stencil_sts_rk2_0(a, b, c, d, dt=f, origin=(0, 0, 0), domain=storage_shape)
df.at["sts_rk2_0", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_sts_rk2_0_defs__gtcuda_83665f6809_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk2_0_defs__gtcuda_83665f6809_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk2_0_defs__gtcuda_83665f6809_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk2_0_defs__gtcuda_83665f6809_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk2_0_defs__gtcuda_83665f6809_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_st

In [134]:
# sts_rk2_0: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_sts_rk2_0(a, b, c, d, dt=f, origin=default_origin, domain=domain)

1.17 ms ± 756 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [135]:
# sts_rk3ws_0
set_annotations(stencil_sts_rk3ws_0_defs, dtype)
stencil_sts_rk3ws_0 = gtscript.stencil(backend=backend, definition=stencil_sts_rk3ws_0_defs, rebuild=False)
out = %timeit -o stencil_sts_rk3ws_0(a, b, c, d, dt=f, origin=(0, 0, 0), domain=storage_shape)
df.at["sts_rk3ws_0", column] = 1000 * out.average

running build_ext
building '_GT_.__main__.m_stencil_sts_rk3ws_0_defs__gtcuda_13e4bd224e_pyext' extension
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk3ws_0_defs__gtcuda_13e4bd224e_pyext_BUILD/users
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk3ws_0_defs__gtcuda_13e4bd224e_pyext_BUILD/users/subbiali
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk3ws_0_defs__gtcuda_13e4bd224e_pyext_BUILD/users/subbiali/jupyterhub
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__main__/m_stencil_sts_rk3ws_0_defs__gtcuda_13e4bd224e_pyext_BUILD/users/subbiali/jupyterhub/tasmania-prognostic-saturation
creating /users/subbiali/jupyterhub/tasmania-prognostic-saturation/notebooks/.gt_cache/py36_1013/gtcuda/__m

In [136]:
# sts_rk3ws_0: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_sts_rk3ws_0(a, b, c, d, dt=f, origin=default_origin, domain=domain)

1.18 ms ± 553 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Serialization

In [137]:
if serialize:
    df.to_excel(writer, sheet_name=sheetname)
    writer.save()

In [138]:
df

,numpy,numexpr,cupy,gtnumpy,gtx86,gtmc,gtcuda
copy,9.449362,NaN,0.383272,9.594794,5.698168,5.819955,0.698259
copychange,50.337060,20.007233,0.751502,50.932797,5.709338,5.804095,0.688785
abs,49.926181,NaN,0.751421,392.239569,5.711308,5.775692,0.684715
iabs,50.135189,NaN,0.751514,393.190949,4.188860,4.271356,0.575475
add,55.094303,20.875209,0.920538,56.124910,7.492006,7.604230,0.936050
iadd,15.295182,NaN,0.535888,55.475913,5.686629,5.784183,0.840824
sub,54.904362,20.907313,0.920541,55.537502,7.509593,7.607268,0.944390
isub,15.501360,NaN,0.535892,55.636531,5.709430,5.784922,0.841926
mul,55.079714,22.223766,0.920547,55.781595,7.475459,7.607106,0.935196
imul,18.344740,NaN,0.535890,55.760560,5.695319,5.871720,0.841497
